### Importar librerías

In [78]:
# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

---

### Tabla `municipios`
* Trasponemos
* Le ponemos el id (pk)

### Tabla `tiempo` 
* Localización solo id (fk)
* Nueva tabla `cielo`con cada tipo de cielo y reemplazamos `tiempo`
* Reset index para (pk) y fecha en columna
* pd.to_datetime a la fecha

### Tabla `ubicaciones`
* Eliminamos el link
* Reemplazamos el municipio por el id
* Nueva tabla `category`
* Nueva table `closed bucket`
* fsq_id (pk)
* reemplazar municipio (fk)

Municipios

In [83]:
df_municipios = pd.read_csv('data/municipios.csv', index_col=0)

In [84]:
# Tenemos que trasponer el DataFrame para que sea más cómodo
df_municipios = df_municipios.T
df_municipios.head(1)

,0,1,2
acebeda-la,41.0869583,-3.624399,La Acebeda


In [85]:
# Comprobamos que no hay duplicados
df_municipios.duplicated().value_counts()

False    176
Name: count, dtype: int64

In [86]:
# Renombramos para que tenga más sentido
df_municipios = df_municipios.rename(columns={0: 'latitud', 1: 'longitud', 2: 'nombre'})
df_municipios.head(1)

,latitud,longitud,nombre
acebeda-la,41.0869583,-3.624399,La Acebeda


In [87]:
df_municipios = df_municipios.reset_index(names='municipio_id')
df_municipios.head(1)

,municipio_id,latitud,longitud,nombre
0,acebeda-la,41.0869583,-3.624399,La Acebeda


---

Tiempo

In [88]:
df_tiempo = pd.read_csv('data/df_aemet_final.csv', index_col=0)

In [89]:
df_tiempo.head(1)

,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
fecha,,,,,,,,,,,,,,
2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


In [90]:
# Comprobamos que no hay duplicados
df_tiempo.duplicated().value_counts()

False    7884
True        1
Name: count, dtype: int64

In [91]:
# Eliminamos duplicados
df_tiempo.drop_duplicates(inplace=True)

In [92]:
df_tiempo.reset_index(inplace=True)
# Habrá que hacer un pd.to_datetime

In [93]:
# Construimos un df auxiliar para los id de los municipios
df_aux = df_tiempo['localizacion_id'].str.split('-id', expand=True)
df_aux = df_aux.drop_duplicates().set_index(0)
# Creamos un diccionario para convertir
dc_municipios = df_aux.to_dict()[1]
df_aux.head(1)

,1
0,
acebeda-la,28001


In [94]:
# Reemplazamos los id
df_tiempo['municipio_id'] = df_tiempo['localizacion_id'].str.split('-id', expand=True)[1].astype(int)

In [95]:
# Ya no necesitamos la columna de localización
df_tiempo.drop(columns='localizacion_id', inplace=True)

Tabla cielo

In [96]:
# Dataframe (series) del cielo, eliminando duplicados y dejando los índices adecuados
df_cielo = df_tiempo['cielo'].drop_duplicates().reset_index(drop = True)

# Creamos un diccionario e invertimos las claves y valores
dc_cielo = {v: k for k, v in df_cielo.to_dict().items()}

# Creamos lo que será nuestra tabla de cielo
df_cielo = df_cielo.reset_index().rename(columns={'index': 'cielo_id', 'cielo': 'estado'})

In [99]:
# Cambiamos los valores en el df de tiempo
df_tiempo['cielo'] = df_tiempo['cielo'].apply(lambda x: dc_cielo[x])
# Renombramos la columna
df_tiempo.rename(columns={'cielo': 'cielo_id'}, inplace = True)

Ahora en `df_municipios` cambiamos los `municipio_id` por sus valores

In [100]:
df_municipios['municipio_id'] = df_municipios['municipio_id'].apply(lambda x: dc_municipios[x])

---

Ubicaciones

In [128]:
df_ubicaciones = pd.read_csv('data/api_foursquare.csv', index_col=0)
df_ubicaciones.head(1)

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


In [129]:
# Comprobamos que no hay duplicados
df_ubicaciones.duplicated().value_counts()

False    907
Name: count, dtype: int64

Nos interesan todas las columnas salvo el link, que contiene información redundante con fsq_id

In [130]:
# Eliminamos la columna que nos sobra
df_ubicaciones = df_ubicaciones.drop(columns='link')

In [131]:
df_ubicaciones.head(1)

,municipio,category,fsq_id,closed_bucket,distance,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


Creamos la tabla de categorías

In [132]:
# Dataframe (series) de las categorías, eliminando duplicados y dejando los índices adecuados
df_categorias = df_ubicaciones['category'].drop_duplicates().reset_index(drop = True)

# Creamos un diccionario e invertimos las claves y valores
dc_categorias = {v: k for k, v in df_categorias.to_dict().items()}

# Creamos lo que será nuestra tabla de categorías
df_categorias = df_categorias.reset_index().rename(columns={'index': 'category_id'})

In [133]:
# Cambiamos los valores en el df de ubicaciones
df_ubicaciones['category'] = df_ubicaciones['category'].apply(lambda x: dc_categorias[x])
# Renombramos la columna
df_ubicaciones.rename(columns={'category': 'category_id'}, inplace = True)

Creamos la tabla closed_bucket

In [135]:
# Dataframe (series) de las categorías, eliminando duplicados y dejando los índices adecuados
df_closed_bucket = df_ubicaciones['closed_bucket'].drop_duplicates().reset_index(drop = True)

# Creamos un diccionario e invertimos las claves y valores
dc_closed_bucket = {v: k for k, v in df_closed_bucket.to_dict().items()}

# Creamos lo que será nuestra tabla de closed bucket
df_closed_bucket = df_closed_bucket.reset_index().rename(columns={'index': 'closed_bucket_id'})

In [137]:
# Cambiamos los valores en el df de ubicaciones
df_ubicaciones['closed_bucket'] = df_ubicaciones['closed_bucket'].apply(lambda x: dc_closed_bucket[x])
# Renombramos la columna
df_ubicaciones.rename(columns={'closed_bucket': 'closed_bucket_id'}, inplace = True)

Nos falta cambiar el municipio por el id correspondiente

In [139]:
df_ubicaciones['municipio'] = df_ubicaciones['municipio'].apply(lambda x: dc_municipios[x])
# Renombramos la columna
df_ubicaciones.rename(columns={'municipio': 'municipio_id'}, inplace = True)